<a href="https://colab.research.google.com/github/JamesForgetterOfWinConditions/MTGOCR/blob/main/SorceryProxies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import urllib.parse

# Load HAR file which is manual network download of fully loaded curiosa.io/cards page
with open("curiosa.io1.har", "r", encoding="utf-8") as f:
    har_data = json.load(f)

urls = []
for entry in har_data.get("log", {}).get("entries", []):
    url = entry.get("request", {}).get("url", "")
    if "curiosa.io/_next/image?url=" in url:
        # Extract and decode the original CloudFront image URL
        decoded = urllib.parse.unquote(url.split("url=")[1].split("&")[0])
        if decoded not in urls:
            urls.append(decoded)

# Save to file
with open("card_image_urls.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(urls))

print(f"Found {len(urls)} card image URLs and saved them to card_image_urls.txt")


Found 651 card image URLs and saved them to card_image_urls.txt


In [ ]:
# Creates database of all cards
import os
import re
import urllib.parse

# Path to your file
file_path = "card_image_urls.txt"

# Output dictionary
cards = {}

# Read all URLs
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        url = line.strip()
        if not url:
            continue

        # Extract filename part
        parsed_url = urllib.parse.urlparse(url)
        filename = os.path.basename(parsed_url.path)  # e.g. avatar_of_air_d_s.png

        # Clean the name (remove suffixes and underscores)
        name = re.sub(r"[_-](b|d|s)?(_[a-z])?\.png$", "", filename, flags=re.IGNORECASE)
        name = name.replace("_", " ").title()

        cards[name] = url

# Save to a CSV for convenience
with open("card_name_url_pairs.csv", "w", encoding="utf-8") as f:
    f.write("name,url\n")
    for name, url in cards.items():
        f.write(f'"{name}","{url}"\n')

print(f"Extracted {len(cards)} cards → saved to card_name_url_pairs.csv")


Extracted 651 cards → saved to card_name_url_pairs.csv


In [ ]:
# Create proxyPDF from database and decklist.txt
# Decklist.txt needs to have special characters removed to match the database (ex ! and ')
import csv
import os
import re
import requests
from io import BytesIO
from PIL import Image
from fpdf import FPDF

# === CONFIG ===
CSV_PATH = "card_name_url_pairs.csv"  # your CSV of name→URL
DECK_PATH = "decklist.txt"            # your formatted decklist
OUTPUT_DIR = "proxy_images"
PDF_PATH = "proxies.pdf"

# === STEP 1: Load card name → URL mapping ===
card_map = {}
with open(CSV_PATH, encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        card_map[row["name"].strip().lower()] = row["url"].strip()

# === STEP 2: Read decklist ===
deck = []
with open(DECK_PATH, encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        match = re.match(r"(\d+)\s+(.+)", line)
        if match:
            count = int(match.group(1))
            name = match.group(2).strip().lower()
            deck.append((name, count))
        else:
            deck.append((line.lower(), 1))

print(f"Loaded {len(deck)} deck entries from {DECK_PATH}")

# === STEP 3: Download and prepare images ===
os.makedirs(OUTPUT_DIR, exist_ok=True)
images = []

for name, count in deck:
    url = None

    # Find best match (exact or partial)
    if name in card_map:
        url = card_map[name]
    else:
        for k in card_map:
            if name in k:
                url = card_map[k]
                break

    if not url:
        print(f"⚠️  Card not found: {name}")
        continue

    print(f"Downloading {name} x{count}")
    try:
        img_data = requests.get(url, timeout=20).content
        img = Image.open(BytesIO(img_data)).convert("RGB")
        for _ in range(count):
            images.append(img)
    except Exception as e:
        print(f"Error downloading {name}: {e}")

# === STEP 4: Generate proxy PDF (Letter size) ===
if not images:
    raise SystemExit("No card images found! Check decklist or CSV mapping.")

pdf = FPDF(orientation="P", unit="mm", format="Letter")
pdf.add_page()

# Card layout: 3 columns × 3 rows (≈63×88 mm per card)
card_width, card_height = 63, 88
cols, rows = 3, 3
x, y = 10, 10
margin_x, margin_y = 10, 10

for i, img in enumerate(images):
    tmp_path = os.path.join(OUTPUT_DIR, f"tmp_{i}.jpg")
    img_resized = img.resize(
        (int(card_width / 25.4 * 300), int(card_height / 25.4 * 300))
    )
    img_resized.save(tmp_path, "JPEG")
    pdf.image(tmp_path, x, y, card_width, card_height)

    # Position next card
    x += card_width + .2
    if (i + 1) % cols == 0:
        x = margin_x
        y += card_height + .2
    if (i + 1) % (cols * rows) == 0 and i + 1 < len(images):
        pdf.add_page()
        x, y = margin_x, margin_y

pdf.output(PDF_PATH)
print(f"✅ Created printable proxy sheet: {PDF_PATH}")


Loaded 46 deck entries from decklist.txt
✅ Created printable proxy sheet: proxies.pdf


In [ ]:
pip install pillow fpdf requests


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=ef6d88f3fb828af28ae21dc8a92f4a5fd55128b619f699b463443d32417b60df
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


<frozen posixpath>:82: RuntimeWarning: coroutine 'main' was never awaited
